Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 32.33 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [9]:
import csv
data={}
with open('facial_expressions/data/legend.csv') as f:
  reader=csv.reader(f)
  next(reader) #skip the frist row
  for row in reader:
    key=row[2].lower()
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]

In [11]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [12]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'master_data'

In [5]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [13]:
from shutil import copyfile
split_size=0.8

for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source=os.path.join('facial_expressions/images',image)
    dest=os.path.join('master_data/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    source=os.path.join('facial_expressions/images',image)
    dest=os.path.join('master_data/testing',emotion,image)
    copyfile(source,dest)

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

In [5]:
model=tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(8,activation='softmax')
])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 16928)             0         
                                                                 
 dense (Dense)               (None, 1024)              1

In [6]:
train_dir='master_data/training'
test_dir='master_data/testing'

train_datagen=ImageDataGenerator(1.0/255)
train_generator=train_datagen.flow_from_directory(
                                                      train_dir,
                                                      target_size=(100,100),
                                                      class_mode='categorical',
                                                      batch_size=128

                                                     )
test_datagen=ImageDataGenerator(1.0/255)
test_generator=test_datagen.flow_from_directory(
                                                      test_dir,
                                                      target_size=(100,100),
                                                      class_mode='categorical',
                                                      batch_size=128

                                                     )

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [7]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)



In [8]:
model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])

<ipython-input-8-1e7c755c9470>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/10
86/86 [==============================] - 24s 231ms/step - loss: 35.0972 - acc: 0.6426 - val_loss: 3.2703 - val_acc: 0.6036
Epoch 2/10
86/86 [==============================] - 18s 215ms/step - loss: 0.5576 - acc: 0.7978 - val_loss: 2.6530 - val_acc: 0.6313
Epoch 3/10
86/86 [==============================] - 20s 230ms/step - loss: 0.3992 - acc: 0.8564 - val_loss: 2.5692 - val_acc: 0.6470
Epoch 4/10
86/86 [==============================] - 18s 215ms/step - loss: 0.2871 - acc: 0.8956 - val_loss: 3.2097 - val_acc: 0.6451
Epoch 5/10
86/86 [==============================] - 19s 224ms/step - loss: 0.2106 - acc: 0.9249 - val_loss: 3.3646 - val_acc: 0.6368


22/22 [==============================] - 4s 177ms/step


array([[2.5019644e-06, 8.9546887e-07, 9.0089196e-01, ..., 7.7128723e-02,
        6.5837880e-03, 6.4318905e-05],
       [1.1760825e-09, 2.1979561e-10, 9.9973315e-01, ..., 6.7798538e-08,
        6.1462553e-05, 1.4701276e-08],
       [1.1747216e-04, 8.1852795e-06, 3.5722234e-04, ..., 9.8801291e-01,
        1.4604822e-04, 1.6936575e-03],
       ...,
       [8.0409351e-05, 6.2082814e-05, 1.0477740e-03, ..., 9.5903456e-01,
        1.1373289e-03, 1.1767808e-04],
       [6.9454032e-08, 8.3594152e-08, 1.2523109e-05, ..., 6.6329122e-01,
        3.3057981e-06, 2.6060317e-08],
       [3.2741283e-03, 1.3340109e-04, 2.6507539e-01, ..., 2.5822141e-03,
        7.1808600e-01, 8.0682160e-03]], dtype=float32)